In [1]:
import pandas as pd
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
df_hoaxPP = pd.read_csv(r'../PrepareHoaxData/filteredPP_manual.csv', encoding='ISO-8859-1')
df_hoaxKesehatan = pd.read_csv(r'../PrepareHoaxData/filteredKesehatan_manual.csv', encoding='ISO-8859-1')

In [3]:
df_hoaxPP['Klasifikasi Berita'] = 'Hoax'
df_hoaxKesehatan['Klasifikasi Berita'] = 'Hoax'

In [4]:
df_hoaxPP.head()

,id,authors,status,classification,title,content,references,tanggal,conclusion,Predict_PP,Klasifikasi Berita
0,20305,36,Salah,Manipulated Content,[SALAH] Mobil Presiden Jokowi Parkir di depan ...,BAPAK JOKOWI MEMANG HARUS DATANG LANGSUNG KEPO...,https://turnbackhoax.id/2024/05/31/salah-mobil...,5/31/2024,"Faktanya, foto mobil Presiden Jokowi itu di d...",Politik & Pemerintahan,Hoax
1,20296,36,Salah,Manipulated Content,[SALAH] USU Kepung Kantor Prabowo Imbas Kenaik...,ISTANA MENCEK4MM!! GILIRAN USU KEPUNG KANTOR P...,[1] https://medan.viva.co.id/sumut/5485-uang-u...,5/31/2024,Unggahan video sebuah akun youtube dengan klai...,Politik & Pemerintahan,Hoax
2,20289,36,Salah,Manipulated Content,[SALAH] Sri Mulyani Berhasil Bongkar Korupsi B...,DHUAARR!! SRIMUL BERHASIL BONGKAR KORUPSI BESA...,https://www.youtube.com/shorts/wefBMbyQa5Q,5/31/2024,Unggahan video sebuah akun youtube dengan klai...,Politik & Pemerintahan,Hoax
3,20288,36,Salah,Misleading Content,"[SALAH] MPR Batalkan Pelantikan Gibran, Prabow...","DHUAARRâ¼ï¸MPR BATALKAN PELANTIKAN GIBRAN, P...",[1] https://rmol.id/hukum/read/2024/05/02/6189...,5/31/2024,Unggahan video sebuah akun youtube dengan klai...,Politik & Pemerintahan,Hoax
4,20287,36,Salah,Manipulated Content,[SALAH] AHY Nekat Bongkar Semua Korupsi di Lin...,RAPAT ISTANA RICUH!! AHY SENEKAT INI BONGKAR S...,[1] https://www.westjavatoday.com/kabulkan-per...,5/31/2024,Unggahan video sebuah akun youtube dengan klai...,Politik & Pemerintahan,Hoax


In [5]:
df_hoaxKesehatan.head()

,id,authors,status,classification,title,content,references,tanggal,conclusion,Predict_Kesehatan,Klasifikasi Berita
0,20334,36,Salah,Misleading Content,[SALAH] Penerima Vaksin COVID-19 mRNA akan Men...,"âProf. Dolores Cahill, University college Du...",https://turnbackhoax.id/2024/06/03/salah-pener...,6/3/2024,Konten yang menyesatkan. Tidak ada bukti ilmia...,Kesehatan,Hoax
1,20275,36,Salah,Misleading Content,[SALAH] PEMBAYARAN BPJS BERDASARKAN GAJI,"""Ini konsep apaan ? \nPembayaran iuran pelayan...",https://www.suara.com/bisnis/2024/05/16/102809...,5/31/2024,"Faktanya, perhitungan baru perihal iuran BPJS ...",Kesehatan,Hoax
2,20261,36,Salah,Misleading Content,[PENIPUAN]: âDokter Tirta Menjelaskan Metode...,Dokter Tirta Menjelaskan Metode Menurunkan Be...,https://www.instagram.com/reel/C68QiMcOxcf/?ig...,5/31/2024,Hasil periksa fakta Yudho Ardi\r\n\r\nInformas...,Kesehatan,Hoax
3,20247,36,Salah,Impostor Content,[SALAH] RS Sri Ratu Medan Resmi Tutup Setelah ...,âCemana menurut klen we para netizen:scream:...,https://www.instagram.com/reel/C7V7J6cO329/ (...,5/30/2024,Hasil periksa fakta Moch. Marcellodiansyah\r\n...,Kesehatan,Hoax
4,20241,36,Salah,Fabricated Content,[SALAH] Mandi di Tiga Waktu Tertentu Bisa Meny...,Mandi pada tiga waktu ini ternyata dapat berak...,https://cekfakta.tempo.co/fakta/2925/keliru-ko...,5/30/2024,Klaim yang mengatakan mandi di tiga waktu tert...,Kesehatan,Hoax


In [6]:
df_hoaxPP['status'].value_counts()

Salah    3390
FALSE       8
Name: status, dtype: int64

In [7]:
df_hoaxPP['classification'].value_counts()

Misleading Content     1443
Manipulated Content     976
False Context           544
Fabricated Content      206
Impostor Content         91
False Connection         86
Satire                   36
Name: classification, dtype: int64

In [8]:
df_hoaxKesehatan['status'].value_counts()

Salah    1068
FALSE       4
Name: status, dtype: int64

In [9]:
df_hoaxKesehatan['classification'].value_counts()

Misleading Content     729
False Context          128
Fabricated Content     111
Manipulated Content     58
Impostor Content        16
False Connection        16
Satire                  11
Name: classification, dtype: int64

### Hilangkan Kategori yang tidak diperlukan (Impostor Content, False Connection, Satire, Clarification)

In [10]:
df_hoaxKesehatan = df_hoaxKesehatan[~df_hoaxKesehatan['classification'].isin(['Impostor Content', 'False Connection', 'Satire'])]
df_hoaxPP = df_hoaxPP[~df_hoaxPP['classification'].isin(['Impostor Content', 'False Connection', 'Satire'])]

In [11]:
print(df_hoaxKesehatan.shape)
df_hoaxKesehatan['classification'].value_counts()

(1029, 11)


Misleading Content     729
False Context          128
Fabricated Content     111
Manipulated Content     58
Name: classification, dtype: int64

In [12]:
print(df_hoaxPP.shape)
df_hoaxPP['classification'].value_counts()

(3185, 11)


Misleading Content     1443
Manipulated Content     976
False Context           544
Fabricated Content      206
Name: classification, dtype: int64

### Convert dataframe hoax hanya menyisakan kolom 'content', dan 'Klasifikasi Berita'

In [47]:
df_hoaxPP = df_hoaxPP[['content', 'Klasifikasi Berita']]
df_hoaxKesehatan = df_hoaxKesehatan[['content', 'Klasifikasi Berita']]

In [48]:
df_hoaxPP['content'] = df_hoaxPP['content'].astype(str)
df_hoaxKesehatan['content'] = df_hoaxKesehatan['content'].astype(str)

### Bersihkan Noise karakter non-ASCII, kata 'NARASI:' & ':(jenis emoticon):, new line(\n & \r), spasi awal & akhir teks, link, dan hastag

In [49]:
def noise_remove(text):
    text = re.sub(r'[^\x00-\x7F]', ' ', text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'http[s]?://\S+|www\.\S+|#\S+', ' ', text)
    text = re.sub('NARASI:', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r':\w+?:', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [50]:
df_hoaxPP['content'] = df_hoaxPP['content'].apply(noise_remove)
df_hoaxKesehatan['content'] = df_hoaxKesehatan['content'].apply(noise_remove)

In [51]:
df_hoaxPP.to_csv('result1_PP.csv', index=False)
df_hoaxKesehatan.to_csv('result1_Kesehatan.csv', index=False)

### Case Folding

In [52]:
def lower_text(text):
    text = text.lower()

    return text

df_hoaxPP['content'] = df_hoaxPP['content'].apply(lower_text)
df_hoaxKesehatan['content'] = df_hoaxKesehatan['content'].apply(lower_text)

In [53]:
df_hoaxPP.to_csv('result2_PP.csv', index=False)
df_hoaxKesehatan.to_csv('result2_Kesehatan.csv', index=False)

### Punctuation Removal

In [54]:
def punct_removal(text):
    text = re.sub(r'[^\w\s]|_', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    return text

df_hoaxPP['content'] = df_hoaxPP['content'].apply(punct_removal)
df_hoaxKesehatan['content'] = df_hoaxKesehatan['content'].apply(punct_removal)

In [55]:
df_hoaxPP.to_csv('result3_PP.csv', index=False)
df_hoaxKesehatan.to_csv('result3_Kesehatan.csv', index=False)

### Melihat kata-kata yang tidak ada pada KBBI pada masing-masing kalimat

In [56]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [57]:
def baca_kamus(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        kamus = set(file.read().splitlines())
    return kamus

def cek_kata(teks, kamus):
    # Tokenisasi teks menjadi kata-kata
    kata_kata = teks.split()
    # Temukan kata-kata yang tidak ada dalam kamus
    kata_not_in_kbbi = [kata for kata in kata_kata if stemmer.stem(kata.lower()) not in kamus]
    return kata_not_in_kbbi

In [58]:
# https://github.com/damzaky/kumpulan-kata-bahasa-indonesia-KBBI
kamus_kata = baca_kamus('list_1.0.0.txt')

In [59]:
df_hoaxPP['word_not_in_kbbi'] = df_hoaxPP['content'].apply(lambda x: cek_kata(x, kamus_kata))
df_hoaxKesehatan['word_not_in_kbbi'] = df_hoaxKesehatan['content'].apply(lambda x: cek_kata(x, kamus_kata))

In [60]:
df_hoaxPP.to_csv('resultcekword_not_in_kbbi_PP.csv', index=False)
df_hoaxKesehatan.to_csv('resultcekword_not_in_kbbi_Kesehatan.csv', index=False)

### Hitung banyaknya frekuensi kata yang tidak ada dalam KBBI

In [61]:
from collections import Counter

def hitung_frekuensi_word_not_in_kbbi(df):
    counter = Counter()
    for typo_list in df['word_not_in_kbbi']:
        counter.update(typo_list)
    return counter

In [62]:
# Hitung frekuensi word_not_in_kbbi dalam dataframe
frekuensi_not_kbbi_PP = hitung_frekuensi_word_not_in_kbbi(df_hoaxPP)
frekuensi_not_kbbi_Kesehatan = hitung_frekuensi_word_not_in_kbbi(df_hoaxKesehatan)

In [63]:
df_frekuensi_not_kbbi_PP = pd.DataFrame(list(frekuensi_not_kbbi_PP.items()), columns=['word_not_in_kbbi', 'Frekuensi']).sort_values(by=['Frekuensi'], ascending=False, ignore_index=True)
df_frekuensi_not_kbbi_PP.head()

,word_not_in_kbbi,Frekuensi
0,jokowi,857
1,anies,594
2,prabowo,429
3,1,317
4,2,298


In [64]:
df_frekuensi_not_kbbi_Kesehatan = pd.DataFrame(list(frekuensi_not_kbbi_Kesehatan.items()), columns=['word_not_in_kbbi', 'Frekuensi']).sort_values(by=['Frekuensi'], ascending=False, ignore_index=True)
df_frekuensi_not_kbbi_Kesehatan.head()

,word_not_in_kbbi,Frekuensi
0,covid,495
1,19,340
2,1,271
3,2,238
4,dr,206


In [65]:
df_frekuensi_not_kbbi_PP.to_csv('result_freq_not_kbbi_PP.csv', index=False)
df_frekuensi_not_kbbi_Kesehatan.to_csv('result_freq_not_kbbi_Kesehatan.csv', index=False)

## Melihat pengaruh stopword terhadap data hoax

### Hanya Number Removal

In [66]:
df_hoaxPP = df_hoaxPP.drop(columns=['word_not_in_kbbi'])
df_hoaxKesehatan = df_hoaxKesehatan.drop(columns=['word_not_in_kbbi'])

In [67]:
df_hoaxPP.head()

,content,Klasifikasi Berita
0,bapak jokowi memang harus datang langsung kepo...,Hoax
1,istana mencek4mm giliran usu kepung kantor pra...,Hoax
2,dhuaarr srimul berhasil bongkar korupsi besar ...,Hoax
3,dhuaarr mpr batalkan pelantikan gibran prabowo...,Hoax
4,rapat istana ricuh ahy senekat ini bongkar smu...,Hoax


In [68]:
df_hoaxKesehatan.head()

,content,Klasifikasi Berita
0,prof dolores cahill university college dublin ...,Hoax
1,ini konsep apaan pembayaran iuran pelayanan k...,Hoax
2,dokter tirta menjelaskan metode menurunkan ber...,Hoax
4,mandi pada tiga waktu ini ternyata dapat berak...,Hoax
5,info paniai news pada hari ini minggu tanggal ...,Hoax


In [69]:
def remove_digits(text):
    text = re.sub(r'\d+', '', text)

    text = re.sub(r'\s+', ' ', text)

    return text

### Stopword & Number Removal

In [70]:
def remove_stopwords_digits(text):
    text = re.sub(r'\d+', '', text)
    words = word_tokenize(text)

    # Menghapus stopword
    stop_words_nltk = stopwords.words('indonesian')
    filter_words = [word for word in words if word not in stop_words_nltk]

    new_text = ' '.join(filter_words)

    new_text = re.sub(r'\s+', ' ', new_text)

    return new_text.strip()

In [71]:
df_hoaxPP['content_with_stopwords'] = df_hoaxPP['content'].apply(remove_digits)
df_hoaxKesehatan['content_with_stopwords'] = df_hoaxKesehatan['content'].apply(remove_digits)

In [72]:
df_hoaxPP['content'] = df_hoaxPP['content'].apply(remove_stopwords_digits)
df_hoaxKesehatan['content'] = df_hoaxKesehatan['content'].apply(remove_stopwords_digits)

### Ekstrak keyword tiap narasi

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [74]:
def extract_top_keywords_from_corpus(texts, top_n=20):
    # Inisialisasi TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    
    # Fit dan transformasi teks ke matriks TF-IDF
    tfidf_matrix = vectorizer.fit_transform(texts)
    
    # Mendapatkan daftar fitur (kata-kata) yang dihasilkan oleh TF-IDF Vectorizer
    feature_names = vectorizer.get_feature_names_out()
    
    # Fungsi untuk mengambil kata kunci dan bobot dari satu dokumen
    def get_top_keywords_with_scores_for_doc(doc_idx):
        tfidf_scores = tfidf_matrix[doc_idx].toarray()[0]
        # Mendapatkan indeks dari kata-kata dengan skor tertinggi
        top_indices = tfidf_scores.argsort()[::-1][:top_n]
        # Mengambil kata kunci dan bobot mereka
        keywords_and_scores = [(feature_names[i], tfidf_scores[i]) for i in top_indices]
        return keywords_and_scores
    
    # Mengambil kata kunci dan bobot untuk setiap dokumen
    top_keywords_and_scores_list = [get_top_keywords_with_scores_for_doc(i) for i in range(tfidf_matrix.shape[0])]
    
    return top_keywords_and_scores_list

In [75]:
df_hoaxPP['all_keywords_with_scores_with_sw'] = extract_top_keywords_from_corpus(df_hoaxPP['content_with_stopwords'].tolist())
df_hoaxPP['all_keywords_with_scores_without_sw'] = extract_top_keywords_from_corpus(df_hoaxPP['content'].tolist())

In [76]:
df_hoaxKesehatan['all_keywords_with_scores_with_sw'] = extract_top_keywords_from_corpus(df_hoaxKesehatan['content_with_stopwords'].tolist())
df_hoaxKesehatan['all_keywords_with_scores_without_sw'] = extract_top_keywords_from_corpus(df_hoaxKesehatan['content'].tolist())

In [77]:
df_hoaxPP.to_csv('diff_stopword_and_not_PP.csv', index=False)
df_hoaxKesehatan.to_csv('diff_stopword_and_not_Kesehatan.csv', index=False)

### Lihat hasil differencenya

In [78]:
df_new_PP = pd.read_csv('diff_stopword_and_not_PP.csv')
df_new_Kesehatan = pd.read_csv('diff_stopword_and_not_Kesehatan.csv')

In [79]:
pd.set_option('display.max_colwidth', None)

In [80]:
df_new_PP.head()

,content,Klasifikasi Berita,content_with_stopwords,all_keywords_with_scores_with_sw,all_keywords_with_scores_without_sw
0,jokowi langsung kepolsek mobil presiden jokowi dgr parkir d polres cirebon bnr kah kpn,Hoax,bapak jokowi memang harus datang langsung kepolsek mobil presiden jokowi dgr parkir d polres cirebon bnr kah kpn,"[('kpn', 0.3311945609710771), ('kepolsek', 0.3311945609710771), ('dgr', 0.3311945609710771), ('cirebon', 0.31515707912698454), ('bnr', 0.31515707912698454), ('parkir', 0.27636204837643663), ('polres', 0.26753598521769667), ('kah', 0.24894579207911635), ('jokowi', 0.21345603627163195), ('mobil', 0.19733076772144356), ('datang', 0.19666599077896357), ('bapak', 0.19289615893005066), ('memang', 0.19289615893005066), ('langsung', 0.15713620273266493), ('harus', 0.15665383947399045), ('presiden', 0.12460551997475477), ('iki', 0.0), ('ikrar', 0.0), ('ikn', 0.0), ('iku', 0.0)]","[('kepolsek', 0.35664639201605397), ('dgr', 0.35664639201605397), ('kpn', 0.35664639201605397), ('bnr', 0.3393764524978803), ('cirebon', 0.3393764524978803), ('parkir', 0.2976000788015047), ('polres', 0.2880957452398635), ('kah', 0.26807692219422996), ('jokowi', 0.22985982912012162), ('mobil', 0.21249535661232322), ('langsung', 0.16921189646167603), ('presiden', 0.13418127699313742), ('image', 0.0), ('ikhtiar', 0.0), ('iku', 0.0), ('iman', 0.0), ('ikrar', 0.0), ('ikn', 0.0), ('iklim', 0.0), ('iklas', 0.0)]"
1,istana mencekmm giliran usu kepung kantor prabowo imbas biaya ukt besaran,Hoax,istana mencekmm giliran usu kepung kantor prabowo imbas biaya ukt naik besar besaran,"[('usu', 0.36639895394006355), ('mencekmm', 0.3486567647136486), ('imbas', 0.336068481124213), ('ukt', 0.336068481124213), ('kepung', 0.3115810246080017), ('giliran', 0.30058410267138297), ('biaya', 0.2541628676622914), ('besaran', 0.2541628676622914), ('naik', 0.24507706267666127), ('kantor', 0.22299889111743054), ('istana', 0.1961021496442386), ('besar', 0.17689635833815798), ('prabowo', 0.14878006557122855), ('ikhlaaash', 0.0), ('iku', 0.0), ('ikut', 0.0), ('ikutan', 0.0), ('ikutannya', 0.0), ('ikuti', 0.0), ('ika', 0.0)]","[('usu', 0.3843767634762626), ('mencekmm', 0.36576403219387915), ('ukt', 0.3525580891861376), ('imbas', 0.3525580891861376), ('kepung', 0.3268691258846574), ('giliran', 0.31533262662137057), ('biaya', 0.2666336773544891), ('besaran', 0.2666336773544891), ('kantor', 0.23394060246289625), ('istana', 0.20572413971280207), ('prabowo', 0.15608014012892038), ('iku', 0.0), ('hvkvm', 0.0), ('iklim', 0.0), ('ikn', 0.0), ('ikrar', 0.0), ('ikutan', 0.0), ('iklankan', 0.0), ('ikutannya', 0.0), ('ikuti', 0.0)]"
2,dhuaarr srimul berhasil bongkar korupsi besaran jkw istana,Hoax,dhuaarr srimul berhasil bongkar korupsi besar besaran jkw di istana,"[('srimul', 0.4440462037717489), ('dhuaarr', 0.43114475630301946), ('bongkar', 0.3358245800012055), ('besaran', 0.3358245800012055), ('berhasil', 0.3109180876125143), ('korupsi', 0.2846544985544083), ('jkw', 0.2846544985544083), ('istana', 0.25910913992799833), ('besar', 0.23373258961489277), ('di', 0.1043358846269065), ('ikrar', 0.0), ('iku', 0.0), ('ikutannya', 0.0), ('ikut', 0.0), ('ikutan', 0.0), ('iklim', 0.0), ('ikuti', 0.0), ('ikutin', 0.0), ('ikutkan', 0.0), ('ikn', 0.0)]","[('srimul', 0.4593486409631877), ('dhuaarr', 0.446002592036565), ('besaran', 0.34739755258639915), ('bongkar', 0.34739755258639915), ('berhasil', 0.32163274853509344), ('korupsi', 0.29446408041411126), ('jkw', 0.29446408041411126), ('istana', 0.2680383939240848), ('iku', 0.0), ('iki', 0.0), ('ikutan', 0.0), ('ikutannya', 0.0), ('ikrar', 0.0), ('ikn', 0.0), ('ikuti', 0.0), ('iklim', 0.0), ('iklas', 0.0), ('iklankan', 0.0), ('iklan', 0.0), ('ikhtiar', 0.0)]"
3,dhuaarr mpr batalkan pelantikan gibran prabowo cari cawapres,Hoax,dhuaarr mpr batalkan pelantikan gibran prabowo harus cari cawapres baru,"[('dhuaarr', 0.4254578288146909), ('pelantikan', 0.3986421546249986), ('batalkan', 0.3755086731840878), ('cari', 0.36671373172161226), ('mpr', 0.34092608

In [81]:
df_new_Kesehatan.head()

,content,Klasifikasi Berita,content_with_stopwords,all_keywords_with_scores_with_sw,all_keywords_with_scores_without_sw
0,prof dolores cahill university college dublin gelar profesr dlm genetika molekuler dr trinity college dublin gelar phd bidng imunologi dr dublin city university memprediksi org yg vaksin covid mati dlm hnga thn aliansi kebebasan dunia kopenhagen,Hoax,prof dolores cahill university college dublin gelar profesr dlm genetika molekuler dr trinity college dublin gelar phd bidng imunologi dr dublin city university dia memprediksi org yg menggunakan vaksin covid akan mati dlm hnga thn aliansi kebebasan dunia kopenhagen,"[('dublin', 0.49075142696963175), ('gelar', 0.2958630780536517), ('college', 0.28578526715391606), ('university', 0.27755109609455336), ('dlm', 0.19539261702835534), ('profesr', 0.16358380898987723), ('trinity', 0.16358380898987723), ('cahill', 0.16358380898987723), ('bidng', 0.16358380898987723), ('molekuler', 0.16358380898987723), ('hnga', 0.16358380898987723), ('imunologi', 0.16358380898987723), ('phd', 0.16358380898987723), ('kopenhagen', 0.16358380898987723), ('memprediksi', 0.15442781801032807), ('dolores', 0.15442781801032807), ('kebebasan', 0.15442781801032807), ('city', 0.14793153902682585), ('aliansi', 0.14793153902682585), ('genetika', 0.14289263357695803)]","[('dublin', 0.4947805829574041), ('gelar', 0.2982921662375856), ('college', 0.28813161472845217), ('university', 0.2798298396687648), ('dlm', 0.19699682496255871), ('kopenhagen', 0.16492686098580137), ('phd', 0.16492686098580137), ('trinity', 0.16492686098580137), ('imunologi', 0.16492686098580137), ('hnga', 0.16492686098580137), ('molekuler', 0.16492686098580137), ('bidng', 0.16492686098580137), ('cahill', 0.16492686098580137), ('profesr', 0.16492686098580137), ('memprediksi', 0.15569569770139097), ('dolores', 0.15569569770139097), ('kebebasan', 0.15569569770139097), ('aliansi', 0.1491460831187928), ('city', 0.1491460831187928), ('genetika', 0.14406580736422608)]"
1,konsep pembayaran iuran pelayanan kesehatan bpjs berdasarkan kualitas pelayanan diterima berdasarkan besaran gaji orang bubarkan bpjs kesehatan,Hoax,ini konsep apaan pembayaran iuran pelayanan kesehatan bpjs kok bukan berdasarkan kualitas pelayanan yang diterima tapi berdasarkan besaran gaji orang kalau seperti ini bubarkan saja bpjs kesehatan,"[('pelayanan', 0.3860272660133271), ('berdasarkan', 0.36668033355967833), ('bpjs', 0.3432563293117587), ('bubarkan', 0.24358905390879124), ('apaan', 0.24358905390879124), ('iuran', 0.24358905390879124), ('konsep', 0.22995506901702345), ('gaji', 0.22995506901702345), ('besaran', 0.2202816027901991), ('pembayaran', 0.2127782794549104), ('kesehatan', 0.19772504623243403), ('kualitas', 0.19301363300666355), ('diterima', 0.19301363300666355), ('kok', 0.16146380585466147), ('kalau', 0.1187181724458399), ('ini', 0.11005830828710125), ('saja', 0.10884029498681735), ('bukan', 0.1085359893430414), ('tapi', 0.10763925502352228), ('seperti', 0.10427389426967475)]","[('pelayanan', 0.42135250265261853), ('berdasarkan', 0.4002351383478012), ('bpjs', 0.37466761066009174), ('iuran', 0.26587981347335554), ('bubarkan', 0.26587981347335554), ('gaji', 0.25099818680847635), ('konsep', 0.25099818680847635), ('besaran', 0.24043950465606767), ('pembayaran', 0.232249554505172), ('kesehatan', 0.2158188045304151), ('kualitas', 0.21067625132630927), ('diterima', 0.21067625132630927), ('orang', 0.0861163223376102), ('idul', 0.0), ('ida', 0.0), ('iedul', 0.0), ('idungnya', 0.0), ('ide', 0.0), ('idi', 0.0), ('idr', 0.0)]"
2,dokter tirta metode menurunkan berat badan diet olahraga,Hoax,dokter tirta menjelaskan metode menurunkan berat badan tanpa diet dan olahraga,"[('tirta', 0.4302629301659173), ('diet', 0.4061805742667628), ('olahraga', 0.3650115149348138), ('metode', 0.32384245560286484), ('menurunkan', 0.31058897876382574), ('menjelaskan', 0.3067557521700704), ('berat', 0.2602642326429008), ('badan', 0.2523332159990823), ('tanpa', 0.2174219810398124), ('dokte

In [82]:
df_hoaxPP = df_hoaxPP.drop(columns=['content_with_stopwords', 'all_keywords_with_scores_with_sw', 'all_keywords_with_scores_without_sw'])
df_hoaxKesehatan = df_hoaxKesehatan.drop(columns=['content_with_stopwords', 'all_keywords_with_scores_with_sw', 'all_keywords_with_scores_without_sw'])

In [83]:
df_hoaxPP.to_csv('endresult_PP.csv', index=False)
df_hoaxKesehatan.to_csv('endresult_Kesehatan.csv', index=False)